In [81]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from scipy.stats import norm
from sklearn import preprocessing
from random import random
import sklearn

In [17]:
df = pd.read_csv("spambase/spambase.data",header=None)


In [18]:
# This data does not have headers so each attribute or field is simply enumerated
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,...,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,0.104553,0.213015,0.280656,0.065425,0.312223,0.095901,0.114208,0.105295,0.090067,0.239413,...,0.038575,0.139030,0.016976,0.269071,0.075811,0.044238,5.191515,52.172789,283.289285,0.394045
std,0.305358,1.290575,0.504143,1.395151,0.672513,0.273824,0.391441,0.401071,0.278616,0.644755,...,0.243471,0.270355,0.109394,0.815672,0.245882,0.429342,31.729449,194.891310,606.347851,0.488698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.065000,0.000000,0.000000,0.000000,0.000000,2.276000,15.000000,95.000000,0.000000
75%,0.000000,0.000000,0.420000,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.160000,...,0.000000,0.188000,0.000000,0.315000,0.052000,0.000000,3.706000,43.000000,266.000000,1.000000
max,4.540000,14.280000,5.100000,42.810000,10.000000,5.880000,7.270000,11.110000,5.260000,18.180000,...,4.385000,9.752000,4.081000,32.478000,6.003000,19.829000,1102.500000,9989.000000,15841.000000,1.000000


In [19]:
# Here df.columns is a list of all the columns and df.columns[0:-1] is all columns minus the last which is y. 
# If the data had headers you could use column names: df[['column1','column2','etc']]
X_train, X_test, Y_train, Y_test = train_test_split(df[df.columns[0:-1]],df[df.columns[-1]], train_size=0.75)

In [20]:
# index for selecting data 0.75 is the percentage in training
index=np.array([1 if random() < 0.75 else 0 for i in range(len(df))])

In [21]:
# Separate both train and test as well as the response variable
X_train=np.array(df[df.columns[0:-1]])[index==1]
X_test=np.array(df[df.columns[0:-1]])[index==0]
Y_train=np.array(df[df.columns[-1]])[index==1]
Y_test=np.array(df[df.columns[-1]])[index==0]

In [22]:
# Normalizar no ayuda mucho pero sale igual al de sklearn. Para que las alturas del pdf signifiquen lo mismo 
scaler = preprocessing.StandardScaler().fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

In [66]:
Y_test_predictions=list()
i=0
for k in X_test:
    if i%20==0:
        print i
    p1=0
    p0=0
    for s in range(len(k)):
        [pcero, puno]=probabilidad(k[s], X_train[:,s], Y_train)
        p1=p1+np.log(puno)
        p0=p0+np.log(pcero)
    if p1>p0:
        Y_test_predictions.append(1)
    else:
        Y_test_predictions.append(0)
    i=i+1
        

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180


In [65]:
def probabilidad(buscar, vector, resultados):
    probs=list()
    for res in [0,1]:
        vec1=vector[resultados==res]
        media=np.mean(vec1)
        sd=np.std(vec1)
        p1=norm(media, sd).pdf(buscar)
        probs.append(p1)
    return probs
        

In [78]:
tp=0
tn=0
fp=0
fn=0

for k in range(len(Y_test)):
    if Y_test[k]==1:
        if Y_test_predictions[k]==Y_test[k]:
            tp=tp+1
    if Y_test[k]==0:
        if Y_test_predictions[k]==Y_test[k]:
            tn=tn+1   
    if Y_test[k]==1:
        if Y_test_predictions[k]!=Y_test[k]:
            fn=fn+1
    if Y_test[k]==0:
        if Y_test_predictions[k]!=Y_test[k]:
            fp=fp+1
            
print "True positives: "+str(tp)
print "True negatives: "+str(tn)
print "False positives: "+str(fp)
print "False negatives: "+str(fn)

True positives: 443
True negatives: 528
False positives: 205
False negatives: 20


In [90]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, Y_train)
Y_test_predictions2=clf.predict(X_test)

tp=0
tn=0
fp=0
fn=0

for k in range(len(Y_test)):
    if Y_test[k]==1:
        if Y_test_predictions2[k]==Y_test[k]:
            tp=tp+1
    if Y_test[k]==0:
        if Y_test_predictions2[k]==Y_test[k]:
            tn=tn+1   
    if Y_test[k]==1:
        if Y_test_predictions2[k]!=Y_test[k]:
            fn=fn+1
    if Y_test[k]==0:
        if Y_test_predictions2[k]!=Y_test[k]:
            fp=fp+1
            
print "True positives: "+str(tp)
print "True negatives: "+str(tn)
print "False positives: "+str(fp)
print "False negatives: "+str(fn)

print pd.DataFrame(confusion_matrix(Y_test, Y_test_predictions2))

print pd.DataFrame(confusion_matrix(Y_test, Y_test_predictions))

True positives: 444
True negatives: 527
False positives: 206
False negatives: 19
     0    1
0  527  206
1   19  444
     0    1
0  528  205
1   20  443
